Here, I build a initial version of the sentiment analysis model

In [54]:
import torch as pt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import json

from torchtext.data.utils import get_tokenizer
import pytorch_lightning as pl

In [ ]:
"""
Plan:
* pytorch LSTM
* pytorch transformer model
"""

# 1. Parse and clean data into dataframe

In [16]:
# Use example code from
# https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV#scrollTo=7igYuRaV4bF7

data = []
with gzip.open("reviews_Office_Products_5.json.gz") as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(f"Total num items in dataset: {len(data)}")

df = pd.DataFrame.from_dict(data)
df = df.loc[:, ["reviewText", "overall"]]

Total num items in dataset: 53258


In [43]:
# manual mapping from "overall rating" to "sentiment"
MIN_RATING = 1
MAX_RATING = 5
NEUTRAL_RATING = 3.5
def rating_to_sentiment(rating, neutral=NEUTRAL_RATING):
    """
    Shifts scale to have `neutral` correspond to 0 sentiment, 1 to correspond to -1 sentiment,
    and 5 to correspond to +1 sentiment. The mapping is a piecewise function, linear from min rating to neutral,
    and again linear from neutral rating to max.
    """
    assert MIN_RATING <= rating <= MAX_RATING
    if rating <= neutral:
        d = neutral - MIN_RATING
        return -1 + (rating - MIN_RATING) / d
    else:
        d = MAX_RATING - neutral
        return (rating - neutral) / d

assert rating_to_sentiment(3.5) == 0
assert rating_to_sentiment(5) == 1
assert rating_to_sentiment(1) == -1
assert rating_to_sentiment(3, neutral=3) == 0
assert rating_to_sentiment(4, neutral=3) == 0.5
assert rating_to_sentiment(2, neutral=3) == -0.5
assert rating_to_sentiment(4) < rating_to_sentiment(4, neutral=3)
assert rating_to_sentiment(2) < rating_to_sentiment(2, neutral=3)

In [50]:
df["sentiment"] = df.overall.apply(rating_to_sentiment)